In [2]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

In [3]:
polskie = open("../polskie-slowa.txt", encoding='utf8')
slowa = [slowo.replace("\n", "") for slowo in polskie.readlines()]
polskie.close()
print(slowa[:100])
print(len(slowa))
slowa = slowa[:5000]

['harcowałabyś', 'groblowych', 'lonżowaliby', 'zasmagałabyś', 'zdradzań', 'niezamianowanej', 'transowych', 'nagonkowy', 'niezagubionych', 'karykaturką', 'wyhandlowałobyś', 'tessarotypie', 'niehermowi', 'dobiłobym', 'kantaty', 'siorpnijcie', 'rozlewanej', 'wciosałabym', 'równoczłonowego', 'jawię', 'stygmatoidalnej', 'niekaliskiej', 'wysiewach', 'niegodkowską', 'dystansowałobym', 'pokratkowane', 'nieetylujących', 'niedowąchaniem', 'siorbnęliście', 'majówkę', 'kontrujący', 'calówkę', 'niezbudowani', 'nieszkółkarską', 'falbaniaste', 'ogipsowałem', 'wykraczalibyśmy', 'niemikrosomalną', 'nieżółtkowego', 'zrywają', 'szatniarką', 'meteopaty', 'dosłużyłybyśmy', 'ampelografio', 'zlepianymi', 'okrętuję', 'niekonwentualną', 'baklavach', 'kreatywny', 'niebladnącego', 'przedkościelne', 'wiktymologami', 'adypinowe', 'odnawiałbym', 'siar', 'nieusiadań', 'usłaniom', 'doniczka', 'oblepiałem', 'szczypka', 'rzymiankę', 'pomieszkujące', 'tapicerować', 'metalurgów', 'plotowana', 'makaronizacyj', 'wzornikują

In [4]:
longestword = 0

unikalneznaki = []
for slowo in slowa:
    for literka in slowo:
        if literka not in unikalneznaki:
            unikalneznaki.append(literka)
    if len(slowo) > longestword:
        longestword = len(slowo)
            
print(unikalneznaki)
print(longestword)
print(len(unikalneznaki))

['h', 'a', 'r', 'c', 'o', 'w', 'ł', 'b', 'y', 'ś', 'g', 'l', 'n', 'ż', 'i', 'z', 's', 'm', 'd', 'ń', 'e', 'j', 't', 'k', 'u', 'ą', 'p', 'ó', 'ę', 'f', 'v', 'ć', 'ź']
15
33


In [5]:
klasy = {}
klasy[0] = "🈳"
for cnt, znak in enumerate(unikalneznaki):
    klasy[cnt+1] = znak

In [6]:
onehotvectory = {}
for cnt, litera in enumerate(unikalneznaki):
    vec = [0 for __ in range(len(unikalneznaki))]
    vec[cnt] = 1
    onehotvectory[litera] = vec

print(onehotvectory)

{'h': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'a': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'r': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'c': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'o': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'w': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ł': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'b': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'y': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'ś': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

In [6]:
print("ilosc wejsc: " + str(len(unikalneznaki) * longestword))

ilosc wejsc: 495


In [7]:
dataset_real = []
# dataset_real = [onehotvectory[slowo[index]] if index<len(slowo) else [0 for __ in range(len(unikalneznaki))] for slowo in slowa for index in range(longestword)]

#longestword = 5
#slowa = ["siema", "abcd", "abc"]
iloscslow = 0

for cnt, slowo in enumerate(slowa):
    slowoencoded = []
    for index in range(longestword):
        if index<len(slowo):
            slowoencoded.append(onehotvectory[slowo[index]])
            # slowoencoded.append(slowo[index])
        else:
            slowoencoded.append([0 for __ in range(len(unikalneznaki))])
    dataset_real = dataset_real + slowoencoded
    iloscslow = iloscslow + 1
    if len(slowo) < longestword:
        for __ in range(longestword - len(slowo)):
            #dataset_real.append("XD")
            slowoencoded = [slowoencoded[-1]] + slowoencoded[:-1]
            dataset_real = dataset_real + slowoencoded
            iloscslow = iloscslow + 1
            #print(dataset_real[-1] + dataset_real[:-1])
    # dataset_real.append("  ")

# for xx in dataset_real:
#     print(xx)


# dataset_fake = []


# for __ in range(len(slowa)):
#     slowo = []
#     wielkosc = random.randint(3, longestword)
#     for xx in range(longestword):
#         if xx < wielkosc:
#             slowo.append(onehotvectory[list(onehotvectory.keys())[random.randint(0, len(onehotvectory)-1)]])
#         else:
#             slowo.append([0 for __ in range(len(unikalneznaki))])
#     dataset_fake.append(slowo)

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()     
        
        self.fc_in = nn.Linear(495, 400)
        self.fc_h1 = nn.Linear(400, 300)
        self.fc_h2 = nn.Linear(300, 200)
        self.fc_h3 = nn.Linear(200, 100)
        self.fc_h4 = nn.Linear(100, 50)
        self.fc_h5 = nn.Linear(50, 20)
        self.fc_out = nn.Linear(20, 1)
        
    def forward(self, x):      
        x = self.fc_in(x)
        x = F.relu(x)
        
        x = self.fc_h1(x)
        x = F.relu(x)
        
        x = self.fc_h2(x)
        x = F.relu(x)
        
        x = self.fc_h3(x)
        x = F.relu(x)
        
        x = self.fc_h4(x)
        x = F.relu(x)
        
        x = self.fc_h5(x)
        x = F.relu(x)
        
        x = self.fc_out(x)
        x = F.sigmoid(x)
        
        return x

In [16]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.fc_in = nn.Linear(500, 500)
        self.fc_h1 = nn.Linear(500, 500)
        self.fc_h2 = nn.Linear(500, 500)
        self.fc_h3 = nn.Linear(500, 500)
        self.fc_h4 = nn.Linear(500, 500)
        self.fc_out = nn.Linear(500, 33)
        
    def forward(self, x):
        x = self.fc_in (x)
        x = F.relu(x)
        
        x = self.fc_h1(x)
        x = F.relu(x)
        
        x = self.fc_h2(x)
        x = F.relu(x)
        
        x = self.fc_h3(x)
        x = F.relu(x)
        
        x = self.fc_h4(x)
        x = F.relu(x)
        
        x = self.fc_out(x)
        x = F.sigmoid(x)
        
        return x

In [17]:
dataset_real = np.array(dataset_real).reshape(iloscslow, -1)
#dataset_fake = np.array(dataset_fake).reshape(len(slowa), -1)

In [18]:
print(dataset_real.shape)
#print(dataset_fake.shape)

(23632, 495)


In [19]:
discriminator = Discriminator().to("cuda:0")
generator = Generator().to("cuda:0")
#TensorFake = torch.Tensor(dataset_fake).to("cuda:0")
TensorReal = torch.Tensor(dataset_real).to("cuda:0")

#TensorIns = torch.cat((TensorReal, TensorFake), 0).to("cuda:0")

#print(TensorIns.shape)

TensorFakeOuts = torch.zeros(len(dataset_real)).to("cuda:0")
TensorRealOuts = torch.ones(len(dataset_real)).to("cuda:0")

#TensorOuts = torch.cat((TensorRealOuts, TensorFakeOuts), 0).to("cuda:0")

#TensorRealOuts = TensorRealOuts.to("cuda:0")

In [20]:
criterion = nn.BCELoss()
criterion_segment = nn.CrossEntropyLoss()

In [21]:
class Onecycle():
    def __init__(self, optimizer, lrmin, lrmax, epochs, model):
        self.b1 = lrmin
        self.a1 = (lrmax - self.b1) / (epochs/2)
    
        self.b2 = lrmax - ((-self.a1)*epochs/2)
        self.epochs = epochs
        self.model = model
        
        self.optimizer = optimizer
        
    def build(self, epoch):
        lr = 0
        if epoch<=self.epochs/2:
            lr = self.a1*epoch+self.b1
        else:
            lr = (-self.a1)*epoch+self.b2
        
        return self.optimizer(self.model.parameters(), lr=lr)
    
    def get_lr_arr(self):
        lrs = []
        for epoch in range(self.epochs):
            if epoch<=self.epochs/2:
                lr = self.a1*epoch+self.b1
                lrs.append(lr)
            else:
                lr = (-self.a1)*epoch+self.b2
                lrs.append(lr)
        return lrs

In [23]:
optim_discriminator = optim.Adam
optim_generator = optim.Adam

onecycle_discriminator = Onecycle(optim_discriminator, 1e-6, 1e-4, 5000, discriminator)
onecycle_generator = Onecycle(optim_generator, 1e-6, 1e-4, 5000, generator)

In [15]:
for epoch in range(5000):
    optimizer_discriminator = onecycle_discriminator.build(epoch)
    optimizer_generator = onecycle_generator.build(epoch)
    
    
    optimizer_discriminator.zero_grad()
    
    y_ = discriminator(TensorReal)
    lossDreal = criterion(y_, TensorRealOuts)
    
    lossDreal.backward()
    
    
    TensorFakeSeeds = torch.rand(iloscslow, 500).to("cuda:0")
    TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")
    
    for i in range(15):
        w = TensorFakeIns[]
    
    y_ = discriminator(TensorFakeIns)
    lossDfake = criterion(y_, TensorFakeOuts)
    
    lossDfake.backward()
    optimizer_discriminator.step()
    
    
    optimizer_generator.zero_grad()
    
    TensorFakeSeeds = torch.rand(iloscslow, 500).to("cuda:0")
    TensorFakeIns = generator(TensorFakeSeeds).to("cuda:0")

    y_ = discriminator(TensorFakeIns)
    lossG = criterion(y_, TensorRealOuts)

    lossG.backward()
    optimizer_generator.step()
    
    if epoch % 100 == 0:
        print(lossDreal.item(), lossDfake.item(), lossG.item())
        
    if epoch % 1000 == 0:
        print(TensorFakeIns)

C:\Anaconda3\lib\site-packages\torch\nn\functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Anaconda3\lib\site-packages\torch\nn\functional.py:2016: UserWarning: Using a target size (torch.Size([23632])) that is different to the input size (torch.Size([23632, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


0.7101739048957825 0.676959753036499 0.709609866142273
tensor([[0.4952, 0.4951, 0.5089,  ..., 0.4908, 0.5033, 0.5102],
        [0.4947, 0.4949, 0.5099,  ..., 0.4910, 0.5024, 0.5098],
        [0.4949, 0.4950, 0.5090,  ..., 0.4917, 0.5027, 0.5089],
        ...,
        [0.4947, 0.4957, 0.5089,  ..., 0.4917, 0.5031, 0.5102],
        [0.4950, 0.4947, 0.5095,  ..., 0.4912, 0.5025, 0.5098],
        [0.4945, 0.4956, 0.5088,  ..., 0.4908, 0.5034, 0.5104]],
       device='cuda:0', grad_fn=<SigmoidBackward>)
0.7098696827888489 0.6760892868041992 0.7105105519294739
0.7093702554702759 0.6739636063575745 0.7127302885055542
0.7086514830589294 0.6706767678260803 0.7161726951599121
0.7076119184494019 0.6656920313835144 0.7214536666870117
0.706194281578064 0.6563113927841187 0.731536328792572
0.7042956352233887 0.6359648704528809 0.75412517786026
0.7018410563468933 0.6052531003952026 0.7901769280433655
0.6994562745094299 0.6079193949699402 0.7870849370956421
0.6978535652160645 0.6657482385635376 0.7216

KeyboardInterrupt: 

In [16]:
with torch.no_grad():
    slowka = ["konstelacje", "maraszynko", "asdasd"]
    
    test_data = np.array([onehotvectory[slowo[index]] if index<len(slowo) else [0 for __ in range(len(unikalneznaki))] for slowo in slowka for index in range(longestword)]).reshape(len(slowka), -1)
    TensorTest = torch.Tensor(test_data).to("cuda:0")

    y_ = discriminator(TensorTest)
    
    y = y_.cpu().detach().numpy()
    
    for cnt, item in enumerate(y):
        if item < 0.5:
            print("Fake: " + slowka[cnt] + " " + str(item[0]))
        else:
            print("Real: " + slowka[cnt] + " " + str(item[0]))

    #print(y_)

Real: konstelacje 1.0
Real: maraszynko 1.0
Real: asdasd 1.0
